<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/ex11notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 ex11notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo11.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の 統計関数群の中の正規分布モジュール (scipy.stats.norm) と多変量正規分布モジュール (scipy.stats.multivariate_normal)
from scipy.stats import norm, multivariate_normal

# scikit-learn の線形判別分析と二次判別分析のクラス
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# scikit-learn の主成分分析のクラス
from sklearn.decomposition import PCA

In [ ]:
# 人間 vs ほげ星人 (2)
URL = 'https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/humanvshoge2.csv'
dfHoge2 = pd.read_csv(URL)
#dfHoge2

In [ ]:
# フィッシャーのアヤメのデータ
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
dfIris = iris.frame
#dfIris

---
## 判別分析 (2)
---



前回に続き，判別分析について解説します．

---
### 多クラスの場合の判別分析

前回は2クラスの問題に限定して解説していましたが，判別分析は多クラスの（クラス数3以上の）問題にも適用できます．


#### 考え方


データを「クラス$1$」から「クラス$K$」までの $K$ 通りのクラスに判別する問題の場合，次のようにします：
1. クラス $k$ ($k = 1, 2, \ldots, K$) のデータが正規分布 ${\cal N}\left(\mathbf{\mu}_k, \Sigma_k\right)$ に従うと仮定して，パラメータ $\mathbf{\mu}_k, \Sigma_k$ を推定する．
1. クラスが未知のデータ $\mathbf{x}$ が与えられたら，それが ${\cal N}\left(\mathbf{\mu}_k, \Sigma_k\right)$ から得られたとすることの対数尤度 $\log\ell_k(\mathbf{x})$ ($k = 1, 2, \ldots, K$) を求め，どのクラスに対して尤度が最大となるか調べる：

$$
\DeclareMathOperator*{\argmax}{argmax}
k^{*} = \argmax_{k=1,\ldots,K}{\log\ell_k(\mathbf{x})} \qquad (1)
$$

ここで，$\displaystyle \DeclareMathOperator*{\argmax}{argmax} \argmax_x f(x)$ は，$f(x)$ が最大となるときの $x$ の値を返す演算子です．$\displaystyle \DeclareMathOperator*{\argmin}{argmin} \argmin$ というのもあり，

$$
\DeclareMathOperator*{\argmax}{argmin}
\begin{aligned}
\min_{x}\{(x-1)^2 + 2\} &= 2\\
\argmin_{x}\{(x-1)^2 + 2\} &= 1\\
\end{aligned}
$$

ということになります．つまり，式$(1)$ の $k^{*}$ は，尤度最大となったクラスの番号となります．

上記では，分散共分散行列がクラス毎に異なってもよいという設定で説明していますが，前回も説明したように，線形判別分析の場合は，全てのクラスの分散共分散行列が共通である，つまり，
$
\Sigma_1 = \Sigma_2 = \dots = \Sigma_K = \Sigma
$
として考えることになります．

#### 実験: Fisher のアヤメのデータの線形判別分析

Fisher のアヤメのデータにはクラスが3つあります．線形判別分析してみましょう．

In [ ]:
## データの準備

# 被説明変数（'target'列）を除いたものを np.array に
X_iris = dfIris.drop(columns='target').to_numpy()
print('# X_iris の最初の5行と X_iris.shape')
print(X_iris[:5, :], X_iris.shape)
print()

# 'target' 列にクラス番号が格納されている
Y_iris = dfIris['target'].to_numpy(dtype=int)
for k, tn in enumerate(iris.target_names):
    print(f'Class{k}: {tn}')
print()
print('# Y_iris と Y_iris.shape')
print(Y_iris, Y_iris.shape)

In [ ]:
# 線形判別分析
lda = LinearDiscriminantAnalysis()
lda.fit(X_iris, Y_iris)  # パラメータの推定
Yt = lda.predict(X_iris) # 予測
print('# クラス予測結果')
print(Yt)
ncorrect = np.sum(Yt == Y_iris)
print(f'正解数/データ数 = {ncorrect}/{len(X_iris)}')

---
### 二次判別分析

#### 二次判別とは

線形判別分析では，各クラスの正規分布の分散共分散行列が共通である，すなわち
$
\Sigma_1 = \Sigma_2 = \dots = \Sigma_K = \Sigma
$ が成り立つという仮定をおいていました．この仮定をなくして，分散共分散行列はクラスごとに異なってもよいとした場合を考えてみましょう．

データを「クラス$1$」から「クラス$K$」までの $K$ 通りに判別する場合，データの所属するクラスを予測する手続きは次のようになります．

1. クラス $k$ ($k = 1, 2, \ldots, K$) のデータが正規分布 ${\cal N}\left(\mathbf{\mu}_k, \Sigma_k\right)$ に従うと仮定して，パラメータ $\mathbf{\mu}_k, \Sigma_k$ を推定する．
1. クラスが未知のデータ $\mathbf{x}$ が与えられたら，それが ${\cal N}\left(\mathbf{\mu}_k, \Sigma_k\right)$ から得られたとすることの対数尤度 $\log\ell_k(\mathbf{x})$ ($k = 1, 2, \ldots, K$) を求め，尤度が最大となるクラス $k^{*}$ を見つける：

$$
\log{\ell_k(\mathbf{x})} = -\frac{D}{2}\log(2\pi) - \frac{1}{2}\log{|\Sigma_k|} -\frac{1}{2} (\mathbf{x}-\mathbf{\mu}_k)^{\top}\Sigma_k^{-1}(\mathbf{x}-\mathbf{\mu}_k)  \qquad (k = 1, 2, \ldots, K)
$$

$$
\DeclareMathOperator*{\argmax}{argmax}
k^{*} = \argmax_{k=1,\ldots,K}{\log\ell_k(\mathbf{x})}
$$

$K = 2$ の場合，判別関数を $\log\ell_{1}(\mathbf{x}) - \log\ell_{2}(\mathbf{x})$ とすれば，その符号によってクラスの予測値を得ることができます（0以上ならクラス1，さもなくばクラス2）．ただし，判別関数が $\mathbf{x}$ の一次式であった線形判別分析のときと異なり，この場合の判別関数は，$\mathbf{x}$ の二次式となります．そのため，このように分散共分散行列がクラスごとに異なってもよいとした場合の判別分析のことを，**二次判別分析**(Quadratic Discriminant Analysis, QDA) といいます．

判別の境界（判別関数の値が $0$ となる点の集合）は，線形判別では平面でしたが，二次判別では二次曲面となります．


#### 例: 「人間」vs「ほげ星人」の二次判別

2次元2クラスのデータを使って実際にやってみましょう．

In [ ]:
# データの準備
X_hoge = dfHoge2[['Height', 'Weight']].to_numpy()
Y_hoge = (dfHoge2['Class'] == 'Human').to_numpy(dtype=int)
print(X_hoge[:5, :], X_hoge.shape)
print(Y_hoge, Y_hoge.shape)

In [ ]:
# 散布図
xmin, xmax = 0, 250
ymin, ymax = 0, 150
fig, ax = plt.subplots(figsize=(12, 6))
ax.scatter(X_hoge[Y_hoge == 1, 0], X_hoge[Y_hoge == 1, 1], label='Human')
ax.scatter(X_hoge[Y_hoge == 0, 0], X_hoge[Y_hoge == 0, 1], label='Hoge')
ax.set_xlim(xmin, xmax)
ax.set_xlabel('Height')
ax.set_ylim(ymin, ymax)
ax.set_ylabel('Weight')
ax.set_aspect('equal')
ax.legend()
plt.show()

このデータは，前回使っていた「人間」vs「ほげ星人」のデータとは別物です．
前回使ったものでは2つのクラスの分布が似た形をしていましたが，こちらでは，両者の分布の形が少し違っています．線形判別と二次判別の結果の違いを示すため，こちらのデータを使ってみることにします（前回のデータだと，線形判別でも二次判別でもほとんど同じ結果となります）．

In [ ]:
# 線形判別分析モデルのパラメータの推定
lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X_hoge, Y_hoge)

# 二次判別分析モデルのパラメータの推定
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_hoge, Y_hoge)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

## 確率密度と判別結果の描画のためのグリッドデータの準備
xmin, xmax = 0, 250
ymin, ymax = 0, 150
xx, yy = np.mgrid[xmin:xmax:1, ymin:ymax:1]
XX = np.dstack((xx, yy))

##  散布図
for i in [0, 1]:
    ax[i].scatter(X_hoge[Y_hoge == 1, 0], X_hoge[Y_hoge == 1, 1], label='Human')
    ax[i].scatter(X_hoge[Y_hoge == 0, 0], X_hoge[Y_hoge == 0, 1], label='Hoge')
    ax[i].set_xlim(xmin, xmax)
    ax[i].set_xlabel('Height')
    ax[i].set_ylim(ymin, ymax)
    ax[i].set_ylabel('Weight')
    ax[i].set_aspect('equal')
    ax[i].legend()

## 確率密度の等高線
# LDA
zz = multivariate_normal.pdf(XX, mean=lda.means_[1], cov=lda.covariance_)
ax[0].contour(xx, yy, zz, colors='blue')
zz = multivariate_normal.pdf(XX, mean=lda.means_[0], cov=lda.covariance_)
ax[0].contour(xx, yy, zz, colors='red')
# QDA
zz = multivariate_normal.pdf(XX, mean=qda.means_[1], cov=qda.covariance_[1])
ax[1].contour(xx, yy, zz, colors='blue')
zz = multivariate_normal.pdf(XX, mean=qda.means_[0], cov=qda.covariance_[0])
ax[1].contour(xx, yy, zz, colors='red')

## 判別結果で領域を塗り分ける
# LDA
zz = lda.predict(XX.reshape(-1, 2)).reshape((XX.shape[0], XX.shape[1]))
ax[0].contourf(xx, yy, zz,   cmap='Blues',   alpha=0.2)
ax[0].contourf(xx, yy, 1-zz, cmap='Oranges', alpha=0.2)
# QDA
zz = qda.predict(XX.reshape(-1, 2)).reshape((XX.shape[0], XX.shape[1]))
ax[1].contourf(xx, yy, zz,   cmap='Blues',   alpha=0.2)
ax[1].contourf(xx, yy, 1-zz, cmap='Oranges', alpha=0.2)

plt.show()

左が線形判別，右が二次判別の結果です．線形判別では，2クラスの分散共分散行列が共通であると仮定していますので，描かれた2つの同心楕円は同じ形をしています．一方，二次判別では両者は別々ですので，異なる形となっています．

この図には，判別の境界も描いてあります．線形判別では直線であるのに対して，二次判別では放物線となっています．
二次判別で得られる2クラスの境界は，この例では放物線ですが，放物線（面）以外の二次曲線（面）になることもあります（例: https://scikit-learn.org/stable/modules/lda_qda.html ）．


---
### 補足

#### 線形判別 vs 二次判別／正規分布の仮定が成り立たないと？




線形判別分析では判別の境界が平面でしたが，二次判別分析では二次曲面になります．そのため，クラス毎の分散共分散行列が共通であるという条件が成り立たないような場合には，二次判別の方が良い（予測の精度が高い）結果が得られる可能性があります．しかし，実際には，次のようなことに注意しなければなりません．

- 二次判別の方が推定すべきパラメータの数が多い： 分散共分散行列ひとつは $\frac{1}{2}D(D+1)$ 個のパラメータを持ちます．二次判別の場合，これをクラスごとに推定しますので，パラメータ数がさらに $K$ 倍になります．サンプルサイズが小さいデータだと，分散共分散行列の推定精度が悪くて良い結果が得られないことがあります．
- 正規分布の仮定が成り立たない場合，線形判別でも二次判別でも良い結果は得られないことがある： 線形判別も二次判別も，クラスごとのデータが正規分布に従うことを仮定しています．この仮定が満たされないような場合には，どちらの手法でも良い結果が得られないことがあります．

次の例は，データの分布が明らかに正規分布ではないような場合にどうなるかを示しています．

In [ ]:
### 正規分布に従わない人工データ
from sklearn.datasets import make_moons
X_moon, Y_moon = make_moons(n_samples=300, noise=0.2)

### QDA してみる
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_moon, Y_moon)

### 散布図と判別結果
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

## 確率密度と判別結果の描画のためのグリッドデータの準備
xmin, xmax = -1.5, 2.5
ymin, ymax = -1.0, 1.5
xx, yy = np.mgrid[xmin:xmax:0.01, ymin:ymax:0.011]
XX = np.dstack((xx, yy))

##  散布図
for i in [0, 1]:
    ax[i].scatter(X_moon[Y_moon == 1, 0], X_moon[Y_moon == 1, 1], label='Class1')
    ax[i].scatter(X_moon[Y_moon == 0, 0], X_moon[Y_moon == 0, 1], label='Class0')
    ax[i].set_xlim(xmin, xmax)
    ax[i].set_ylim(ymin, ymax)
    ax[i].set_aspect('equal')
    ax[i].legend()

## 確率密度の等高線
zz = multivariate_normal.pdf(XX, mean=qda.means_[1], cov=qda.covariance_[1])
ax[1].contour(xx, yy, zz, colors='blue')
zz = multivariate_normal.pdf(XX, mean=qda.means_[0], cov=qda.covariance_[0])
ax[1].contour(xx, yy, zz, colors='red')

## 判別結果で領域を塗り分ける
zz = qda.predict(XX.reshape(-1, 2)).reshape((XX.shape[0], XX.shape[1]))
ax[1].contourf(xx, yy, zz,   cmap='Blues',   alpha=0.2)
ax[1].contourf(xx, yy, 1-zz, cmap='Oranges', alpha=0.2)

plt.show()

当たり前ですが，あまりうまく判別できていません．

「機械学習I/II」では，このような場合でもうまく判別できるような手法を取り上げます．

#### 事前確率（クラスごとのデータの出現確率）の扱い

ここまで解説してきた判別分析の方法は，クラスごとの正規分布を推定し，それらの対数尤度を基準として判別するというものです．「クラスごとのデータが正規分布に従う」という仮定をおいており，その仮定が成り立たないときにはうまくいかないかもしれない，ということはすでに説明しました．

実は，ここまで説明してきた方法には，隠れた仮定がもうひとつあります．それは，「どのクラスのデータも等しい確率で生起する」というものです．この仮定が成り立たない場合，すなわち，クラスごとのデータの出現確率に偏りがあるような場合，クラスごとの正規分布に対する対数尤度を基準とする，という方法ではよい結果が得られない可能性があります．

これに対処する方法の一つが，クラスごとのデータの出現確率（これを「クラス事前確率」といいます）まで含めてモデル化を行う，というものです．
具体的な定式化等については，「機械学習I/II」で取り上げます．scikit-learn の判別分析のクラスでは，このクラス事前確率を含めたモデルが使われており，データ中の各クラスの出現頻度からその値を推定するようになっています（注）．

※注意: 以下のリンク先のドキュメントの $P(y=k)$ というのが事前確率です：
https://scikit-learn.org/stable/modules/lda_qda.html#mathematical-formulation-of-the-lda-and-qda-classifiers

---
### 【発展】次元削減手法としての線形判別分析

線形判別分析は判別のための手法ですが，実は，次元削減手法の一種として考えることもできます．ここでは，詳しい解説は省いて，直感的にそのことを説明します．

#### 2クラス線形判別はデータを1次元に変換する

2クラス問題に対する線形判別分析は，データを1次元に次元削減する軸を見つける処理とみなせます．

クラス1に属するデータが ${\cal N}(\mathbf{\mu}_1, \Sigma)$ に従い，クラス2に属するデータが ${\cal N}(\mathbf{\mu}_2, \Sigma)$ に従うとき，線形判別分析によって得られる線形判別関数 $z(\mathbf{x})$ は

$$
z(\mathbf{x}) = (\mathbf{\mu}_1 - \mathbf{\mu}_2)^{\top}\Sigma^{-1}\left( \mathbf{x} - \frac{\mathbf{\mu}_1+\mathbf{\mu}_2}{2} \right)
$$

と表せるのでした（ex10notebookAの式$(16)$）．
ここで $\mathbf{w} = \Sigma^{-1}(\mathbf{\mu}_1 - \mathbf{\mu}_2)$ とおくと，

$$
z(\mathbf{x}) = \mathbf{w}^{\top}\left( \mathbf{x} - \frac{\mathbf{\mu}_1+\mathbf{\mu}_2}{2} \right)
$$

と書けます．この式は，$D$ 個の要素から成る $\mathbf{x}$ から一つの実数値 $z(\mathbf{x})$ を求める変換を表しています（注）．$z(\mathbf{x})$ の値は，下図左に示すように，点 $\frac{\mathbf{\mu}_1+\mathbf{\mu}_2}{2}$ を通りベクトル $\mathbf{w}$ に並行な直線上に点 $\mathbf{x}$ を射影したときに，この直線上で測ったその点の位置を表しています．下図右は，$z(\mathbf{x})$ の値を横軸にとってヒストグラムを描いたものです．この軸上で見ると，点Aは正の方に，点Bは負の方にいることになります．

※注意: 主成分分析において $\mathbf{x}$ から主成分スコア $y$ を求める計算（$y = \mathbf{u}^{\top}(\mathbf{x}-\bar{\mathbf{x}}）$) と似た形をしてますね．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/dimreductionLDA.png">

2クラス問題の線形判別分析でデータを1次元に変換できることが分かりましたが，この変換は，主成分分析で得られるものとはどう違うのでしょうか．両者の違いを理解してもらうために，2クラス2次元のデータに対して主成分分析と線形判別分析を適用した結果を例にあげます．

下図左のピンク色の星印は，これらデータ全体の（クラスを区別しないで求めた）平均です．このデータでは，2クラスに等しい数のデータが含まれているので，これは2クラス（オレンジ色と青色）それぞれの平均の中点と一致しています．

このデータに主成分分析を適用して第1主成分を求めると，ピンクの矢線の向きとなります．主成分分析ではデータのクラス分けは考慮されず，データ全体の分散が最も大きくなる軸が選ばれています．一方，
線形判別分析を適用して得られる軸は水色の矢線の向きとなります．

どちらの軸を選んでも2次元のデータを1次元にすることができますが，その軸方向に見たデータの分布は異なります．主成分分析を用いて1次元に変換したデータの値（第1主成分スコア）は，下図真ん中のように分布しており，2つのクラスの値が結構入り混じっています．
これに対して，線形判別分析を用いて1次元に変換したデータの値（判別関数の値）は，下図右のように分布します．こちらの方が2クラスのデータがよりうまく分離されていることが分かります．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/PCAvsLDA.png">


このように，線形判別分析を利用した次元削減では，クラスをよく分離するような軸が選ばれ，主成分分析を利用した次元削減では，データ全体の分散が大きくなるような軸が選ばれます．判別が目的である場合，主成分分析による次元削減は適切ではないこともあります．


#### $K$クラス線形判別による $(K-1)$ 次元への次元削減

上記の説明は，2クラス問題のデータを1次元に変換するという話でした．これを一般化すると，次のことがいえます．

> $K$クラス問題に線形判別分析を適用すると，データを $(K-1)$ 次元に変換する線形変換の中で，クラス間の分離が最もよくなるようなものを見つけることができる．

このことは，数学的にきちんと証明できるのですが，ここではその証明およびこの線形変換の具体的な計算法についての説明は省略します（注）．
変換後の次元数を元の次元数以下で自由に決められる主成分分析と違い，次元数が $(K-1)$ 以下になる（$K$次元目以降は有効ではない）という強い制約があることに注意が必要です．

<span style="font-size: 75%">
※注: 気になるひとは，この授業の「参考情報」のページに上げた「わかりやすいパターン認識 第2版」をどうぞ．
</span>

試しに，Fisherのアヤメのデータを次元削減する実験をやってみましょう．このデータは下図に示すような4次元のデータです．クラス数は3です．主成分分析と線形判別分析のそれぞれを利用して，このデータを2次元に変換してみましょう．

In [ ]:
# 4つの変数のうち2つ選んで散布図を描く
seaborn.pairplot(dfIris, hue='target', palette='tab10', corner=True)

まずは主成分分析です．以前の notebook でやっていたように NumPy 等を使って自分で計算してもいいのですが，ここでは scikit-learn の [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) を使っています．主成分分析ではクラス分けを考慮しませんから，`X` のみを用い，`Y` の方は使いません．



In [ ]:
# 主成分分析による次元削減
pca = PCA(n_components=2)
pca.fit(X_iris)
X_pca = pca.transform(X_iris)
print(X_pca[:5, :], X_pca.shape)

`X_pca` の0列目が第1主成分スコア，1列目が第2主成分スコア，です．

次に，線形判別分析による次元削減です．前回の演習でも使っていた [sklearn.discriminant_analysis.LinearDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html) に次元削減の変換を行う機能も備わっています．

左が主成分分析を利用した変換（上位2つの主成分を用いて2次元に次元削減したもの）の結果，右が線形判別分析を利用した変換の結果です．主成分分析で2次元にした場合，3つのクラスがあまりよく分離されていません．しかし，線形判別分析で2次元にした場合，3つのクラスがよく分離されています（横軸の値だけでほぼ3クラスが判別できる）．

In [ ]:
# 線形判別分析による次元削減
lda = LinearDiscriminantAnalysis()
lda.fit(X_iris, Y_iris)
X_lda = lda.transform(X_iris)
print(X_lda[:5, ], X_lda.shape)

元のアヤメのデータは4次元ですが，2次元に変換すれば散布図に描いてみることができます．描いてみるとこんなんなります．

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

for k in range(3):
    ax[0].scatter(X_pca[Y_iris == k, 0], X_pca[Y_iris == k, 1], label=iris.target_names[k])
    ax[1].scatter(X_lda[Y_iris == k, 0], X_lda[Y_iris == k, 1], label=iris.target_names[k])

for i in [0, 1]:
    ax[i].axhline(0, color='gray')
    ax[i].axvline(0, color='gray')
    ax[i].set_aspect('equal')
    ax[i].legend()

ax[0].set_xlim(-4.5, 4.5)
ax[0].set_ylim(-2.5, 2.5)
ax[1].set_xlim(-10.5, 10.5)
ax[1].set_ylim(-6, 6)

plt.show()